<a href="https://colab.research.google.com/github/LiHantang/Production_Optimization_MSC/blob/main/MSC_prescriptive_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp

     |████████████████████████████████| 14.2 MB 5.0 MB/s 


In [ ]:
from pulp import *
import pandas as pd

In [ ]:
from google.colab import files
f = files.upload()

Saving transportation_distance.csv to transportation_distance.csv


In [ ]:
trans_dist = pd.read_csv('transportation_distance.csv',index_col=[0])
trans_dist

,STL1,STL2,STL3,STL4,KS1,KS2,COL,SPR
STL,10,10,10,10,250,250,120,215
KSC,250,250,250,250,10,10,130,165
CLB,120,120,120,120,130,130,10,170
SPF,215,215,215,215,165,165,170,10
IDP,240,240,240,240,15,15,125,170


# Problem 1

In [ ]:
# Establish linear minimization problem to minimize the total cost
prob = LpProblem('prob', LpMinimize)

# Parameters
demand = {'Shoes':{'STL1':8000, 'STL2':6500, 'STL3':7500, 'STL4':0, 'KS1':9500, 'KS2':2300, 'COL':7000, 'SPR':6000},\
          'Boots':{'STL1':2200, 'STL2':0, 'STL3':1200, 'STL4':900, 'KS1':4500, 'KS2':0, 'COL':2000, 'SPR':3200}}
fixed_cost = {'STL':{1:0,2:2500000},'KSC':{1:1000000,2:1700000},'CLB':{1:1000000,2:1500000}, 'SPF':{1:1000000,2:1500000},'IDP':{1:900000,2:1300000}}
variable_cost = {'STL':{'Shoes':20,'Boots':30},'KSC':{'Shoes':18,'Boots':25},'CLB':{'Shoes':15,'Boots':20}, 'SPF':{'Shoes':15,'Boots':20},'IDP':{'Shoes':12,'Boots':18}}
unit_transportation_cost = {'Shoes':0.03, 'Boots':0.04}
dist = trans_dist.transpose().to_dict()

# Define Variables
plant = ['STL','KSC','CLB','SPF','IDP']
destination = ['STL1','STL2','STL3','STL4','KS1','KS2','COL','SPR']
lines = [1,2]
products = ['Shoes', 'Boots']
decision_combinations = [(i,j) for i in plant for j in lines]
production_combinations = [(i,j,p) for i in plant for j in lines for p in products]
trans_combinations = [(i,k,p) for i in plant for k in destination for p in products]

decision = LpVariable.dicts('decision',decision_combinations,lowBound=0,cat='Binary')
quantity = LpVariable.dicts('quantity',production_combinations,lowBound=0, cat='Continuous')
trans_quantity = LpVariable.dicts('trans_quanity',trans_combinations, lowBound=0, cat='Continuous')

In [ ]:
# Objective function
fix_cost = lpSum([decision[(i,j)]*fixed_cost[i][j] for i in plant for j in lines])
variable_cost = lpSum([quantity[(i,j,p)]* variable_cost[i][p] for i in plant for j in lines for p in products])
trans_cost = lpSum([dist[i][d]*(0.03*trans_quantity[(i,d,'Shoes')]+0.04*trans_quantity[(i,d,'Boots')]) for i in plant for d in destination])
prob += fix_cost + variable_cost + trans_cost

# Constraints
## We already have 2 lines in STL and 1 line in KSC
prob += decision[('STL',2)] == 1, 'Production_STL'
prob += lpSum([decision[('KSC',j)] for j in lines]) == 1, 'Production_KSC'

## Every other plant can have 1, 2 or no product line
for i in plant:
    prob += lpSum([decision[(i,j)] for j in lines]) <= 1, 'Production_Line_%s' %i

## The capacity of the plant should be respected
for i in plant:
    for j in lines:
        prob += lpSum([(1/15000)*quantity[(i,j,'Shoes')]+(1/9000)*quantity[(i,j,'Boots')]]) <= decision[(i,j)]*j, 'Capacity_%s_%s' %(i,j)

## The demand of customeres should be satisfied
for d in destination:
    for p in products:
        prob += lpSum([trans_quantity[(i,d,p)] for i in plant]) == demand[p][d], 'Demand_%s_%s' %(p,d)

## expansion and/or extra
prob += lpSum([decision[(i,j)] for i in plant[2:] for j in lines]) <= 1, 'Extra_Plant_%s_%i' %(i,j)

## The output of the plant should equal to its prouction
for i in plant:
    for p in products:
        prob += lpSum([trans_quantity[(i,d,p)] for d in destination]) == lpSum([quantity[(i,j,p)] for j in lines]), 'Output_%s_%s' %(i,p)

In [ ]:
# Solve for the answer
prob.solve()
print('Status: ', LpStatus[prob.status])

Status:  Optimal


In [ ]:
# get the minimized cost
print("Total cost = ", value(prob.objective))

Total cost =  5990884.0


In [ ]:
# production
output = []
for i in plant:
    for j in lines:
        output_sub = [decision[(i,j)].varValue]
        for p in products:
            output_sub.append(quantity[(i,j,p)].varValue)
        output.append(output_sub)
production_df = pd.DataFrame(output,index=decision_combinations, columns=['Decision','Shoes','Boots'])
production_df

,Decision,Shoes,Boots
"(STL, 1)",0.0,0.0,0.0
"(STL, 2)",1.0,22000.0,4300.0
"(KSC, 1)",1.0,0.0,6580.0
"(KSC, 2)",0.0,0.0,0.0
"(CLB, 1)",0.0,0.0,0.0
"(CLB, 2)",0.0,0.0,0.0
"(SPF, 1)",0.0,0.0,0.0
"(SPF, 2)",0.0,0.0,0.0
"(IDP, 1)",0.0,0.0,0.0
"(IDP, 2)",1.0,24800.0,3120.0


In [ ]:
# transportation shoes
output = {}
for j in destination:
    output[j]=[trans_quantity[(i,j,'Shoes')].varValue for i in plant]
output_df = pd.DataFrame(output, index=plant, columns=destination)
output_df

,STL1,STL2,STL3,STL4,KS1,KS2,COL,SPR
STL,8000.0,6500.0,7500.0,0.0,0.0,0.0,0.0,0.0
KSC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CLB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SPF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IDP,0.0,0.0,0.0,0.0,9500.0,2300.0,7000.0,6000.0


In [ ]:
# transportation boots
output = {}
for j in destination:
    output[j]=[trans_quantity[(i,j,'Boots')].varValue for i in plant]
output_df = pd.DataFrame(output, index=plant, columns=destination)
output_df

,STL1,STL2,STL3,STL4,KS1,KS2,COL,SPR
STL,2200.0,0.0,1200.0,900.0,0.0,0.0,0.0,0.0
KSC,0.0,0.0,0.0,0.0,4500.0,0.0,0.0,2080.0
CLB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SPF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IDP,0.0,0.0,0.0,0.0,0.0,0.0,2000.0,1120.0


# Problem 2

In [ ]:
# Establish linear minimization problem to minimize the total cost
prob2 = LpProblem('prob2', LpMaximize)

# Parameters
demand = {'Shoes':{'STL1':8000, 'STL2':6500, 'STL3':7500, 'STL4':0, 'KS1':9500, 'KS2':2300, 'COL':7000, 'SPR':6000},\
          'Boots':{'STL1':2200, 'STL2':0, 'STL3':1200, 'STL4':900, 'KS1':4500, 'KS2':0, 'COL':2000, 'SPR':3200}}
fixed_cost = {'STL':{1:0,2:2500000},'KSC':{1:1000000,2:1700000},'CLB':{1:1000000,2:1500000}, 'SPF':{1:1000000,2:1500000},'IDP':{1:900000,2:1300000}}
variable_cost = {'STL':{'Shoes':20,'Boots':30},'KSC':{'Shoes':18,'Boots':25},'CLB':{'Shoes':15,'Boots':20}, 'SPF':{'Shoes':15,'Boots':20},'IDP':{'Shoes':12,'Boots':18}}
unit_transportation_cost = {'Shoes':0.03, 'Boots':0.04}
dist = trans_dist.transpose().to_dict()

# Define Variables
plant = ['STL','KSC','CLB','SPF','IDP']
destination = ['STL1','STL2','STL3','STL4','KS1','KS2','COL','SPR']
lines = [1,2]
products = ['Shoes', 'Boots']
decision_combinations = [(i,j) for i in plant for j in lines]
production_combinations = [(i,j,p) for i in plant for j in lines for p in products]
trans_combinations = [(i,k,p) for i in plant for k in destination for p in products]

decision = LpVariable.dicts('decision',decision_combinations,lowBound=0,cat='Binary')
quantity = LpVariable.dicts('quantity',production_combinations,lowBound=0, cat='Continuous')
trans_quantity = LpVariable.dicts('trans_quanity',trans_combinations, lowBound=0, cat='Continuous')

In [ ]:
# Objective function
fix_cost = lpSum([decision[(i,j)]*fixed_cost[i][j] for i in plant for j in lines])
variable_cost = lpSum([quantity[(i,j,p)]* variable_cost[i][p] for i in plant for j in lines for p in products])
trans_cost = lpSum([dist[i][d]*(0.03*trans_quantity[(i,d,'Shoes')]+0.04*trans_quantity[(i,d,'Boots')]) for i in plant for d in destination])
revenue = lpSum([80*quantity[(i,j,'Boots')]+50*quantity[(i,j,'Shoes')] for i in plant for j in lines])
prob2 += revenue - fix_cost - variable_cost - trans_cost

# Constraints
## We already have 2 lines in STL and 1 line in KSC
prob2 += decision[('STL',2)] == 1, 'Production_STL'
prob2 += lpSum([decision[('KSC',j)] for j in lines]) == 1, 'Production_KSC'

## Every other plant can have 1, 2 or no product line
for i in plant:
    prob2 += lpSum([decision[(i,j)] for j in lines]) <= 1, 'Production_Line_%s' %i

## The capacity of the plant should be respected
for i in plant:
    for j in lines:
        prob2 += lpSum([(1/15000)*quantity[(i,j,'Shoes')]+(1/9000)*quantity[(i,j,'Boots')]]) <= decision[(i,j)]*j, 'Capacity_%s_%s' %(i,j)

## The demand of customeres should be satisfied
for d in destination:
    for p in products:
        prob2 += lpSum([trans_quantity[(i,d,p)] for i in plant]) == demand[p][d], 'Demand_%s_%s' %(p,d)

## expansion and/or extra
prob2 += lpSum([decision[(i,j)] for i in plant[2:] for j in lines]) <= 1, 'Extra_Plant_%s_%i' %(i,j)

## The output of the plant should equal to its prouction
for i in plant:
    for p in products:
        prob2 += lpSum([trans_quantity[(i,d,p)] for d in destination]) == lpSum([quantity[(i,j,p)] for j in lines]), 'Output_%s_%s' %(i,p)

In [ ]:
# Solve for the answer
prob2.solve()
print('Status: ', LpStatus[prob2.status])

Status:  Optimal


In [ ]:
# get the minimized cost
print("Total profit = ", value(prob2.objective))

Total profit =  -2530884.0


In [ ]:
# production
output = []
for i in plant:
    for j in lines:
        output_sub = [decision[(i,j)].varValue]
        for p in products:
            output_sub.append(quantity[(i,j,p)].varValue)
        output.append(output_sub)
production_df = pd.DataFrame(output,index=decision_combinations, columns=['Decision','Shoes','Boots'])
production_df

,Decision,Shoes,Boots
"(STL, 1)",0.0,0.0,0.0
"(STL, 2)",1.0,22000.0,4300.0
"(KSC, 1)",1.0,0.0,6580.0
"(KSC, 2)",0.0,0.0,0.0
"(CLB, 1)",0.0,0.0,0.0
"(CLB, 2)",0.0,0.0,0.0
"(SPF, 1)",0.0,0.0,0.0
"(SPF, 2)",0.0,0.0,0.0
"(IDP, 1)",0.0,0.0,0.0
"(IDP, 2)",1.0,24800.0,3120.0


In [ ]:
# transportation shoes
output = {}
for j in destination:
    output[j]=[trans_quantity[(i,j,'Shoes')].varValue for i in plant]
output_df = pd.DataFrame(output, index=plant, columns=destination)
output_df

,STL1,STL2,STL3,STL4,KS1,KS2,COL,SPR
STL,8000.0,6500.0,7500.0,0.0,0.0,0.0,0.0,0.0
KSC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CLB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SPF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IDP,0.0,0.0,0.0,0.0,9500.0,2300.0,7000.0,6000.0


In [ ]:
# transportation boots
output = {}
for j in destination:
    output[j]=[trans_quantity[(i,j,'Boots')].varValue for i in plant]
output_df = pd.DataFrame(output, index=plant, columns=destination)
output_df

,STL1,STL2,STL3,STL4,KS1,KS2,COL,SPR
STL,2200.0,0.0,1200.0,900.0,0.0,0.0,0.0,0.0
KSC,0.0,0.0,0.0,0.0,4500.0,0.0,0.0,2080.0
CLB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SPF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IDP,0.0,0.0,0.0,0.0,0.0,0.0,2000.0,1120.0


# Problem 3

In [ ]:
# Establish linear minimization problem to minimize the total cost
prob3 = LpProblem('prob3', LpMaximize)

# Parameters
demand = {'Shoes':{'STL1':8000, 'STL2':6500, 'STL3':7500, 'STL4':0, 'KS1':9500, 'KS2':2300, 'COL':7000, 'SPR':6000},\
          'Boots':{'STL1':2200, 'STL2':0, 'STL3':1200, 'STL4':900, 'KS1':4500, 'KS2':0, 'COL':2000, 'SPR':3200}}
fixed_cost = {'STL':{1:0,2:2500000},'KSC':{1:1000000,2:1700000},'CLB':{1:1000000,2:1500000}, 'SPF':{1:1000000,2:1500000},'IDP':{1:900000,2:1300000}}
variable_cost = {'STL':{'Shoes':20,'Boots':30},'KSC':{'Shoes':18,'Boots':25},'CLB':{'Shoes':15,'Boots':20}, 'SPF':{'Shoes':15,'Boots':20},'IDP':{'Shoes':12,'Boots':18}}
unit_transportation_cost = {'Shoes':0.03, 'Boots':0.04}
dist = trans_dist.transpose().to_dict()

# Define Variables
plant = ['STL','KSC','CLB','SPF','IDP']
destination = ['STL1','STL2','STL3','STL4','KS1','KS2','COL','SPR']
lines = [1,2]
products = ['Shoes', 'Boots']
decision_combinations = [(i,j) for i in plant for j in lines]
production_combinations = [(i,j,p) for i in plant for j in lines for p in products]
trans_combinations = [(i,k,p) for i in plant for k in destination for p in products]

decision = LpVariable.dicts('decision',decision_combinations,lowBound=0,cat='Binary')
quantity = LpVariable.dicts('quantity',production_combinations,lowBound=0, cat='Continuous')
trans_quantity = LpVariable.dicts('trans_quanity',trans_combinations, lowBound=0, cat='Continuous')

In [ ]:
# Objective function
fix_cost = lpSum([decision[(i,j)]*fixed_cost[i][j] for i in plant for j in lines])
variable_cost = lpSum([quantity[(i,j,p)]* variable_cost[i][p] for i in plant for j in lines for p in products])
trans_cost = lpSum([dist[i][d]*(0.03*trans_quantity[(i,d,'Shoes')]+0.04*trans_quantity[(i,d,'Boots')]) for i in plant for d in destination])
revenue = lpSum([80*quantity[(i,j,'Boots')]+50*quantity[(i,j,'Shoes')] for i in plant for j in lines])
prob3 += revenue - fix_cost - variable_cost - trans_cost

# Constraints
## We already have 2 lines in STL and 1 line in KSC
prob3 += decision[('STL',2)] == 1, 'Production_STL'
prob3 += lpSum([decision[('KSC',j)] for j in lines]) == 1, 'Production_KSC'

## Every other plant can have 1, 2 or no product line
for i in plant:
    prob3 += lpSum([decision[(i,j)] for j in lines]) <= 1, 'Production_Line_%s' %i

## The demand of customeres can be unsatisfied
for d in destination:
    for p in products:
        prob3 += lpSum([trans_quantity[(i,d,p)] for i in plant]) <= demand[p][d], 'Demand_%s_%s' %(p,d)

## The capacity of the plant should be respected
for i in plant:
    for j in lines:
        prob3 += lpSum([(1/15000)*quantity[(i,j,'Shoes')]+(1/9000)*quantity[(i,j,'Boots')]]) <= decision[(i,j)]*j, 'Capacity_%s_%s' %(i,j)

## expansion and/or extra
prob3 += lpSum([decision[(i,j)] for i in plant[2:] for j in lines]) <= 1, 'Extra_Plant_%s_%i' %(i,j)

## The output of the plant should equal to its prouction
for i in plant:
    for p in products:
        prob3 += lpSum([trans_quantity[(i,d,p)] for d in destination]) == lpSum([quantity[(i,j,p)] for j in lines]), 'Output_%s_%s' %(i,p)

In [ ]:
# Solve for the answer
prob3.solve()
print('Status: ', LpStatus[prob3.status])

Status:  Optimal


In [ ]:
# get the minimized cost
print("Total profit = ", value(prob3.objective))

Total profit =  -2127270.0


In [ ]:
# production
output = []
for i in plant:
    for j in lines:
        output_sub = [decision[(i,j)].varValue]
        for p in products:
            output_sub.append(quantity[(i,j,p)].varValue)
        output.append(output_sub)
production_df = pd.DataFrame(output,index=decision_combinations, columns=['Decision','Shoes','Boots'])
production_df

,Decision,Shoes,Boots
"(STL, 1)",0.0,0.0,0.0
"(STL, 2)",1.0,22000.0,4800.0
"(KSC, 1)",1.0,7500.0,4500.0
"(KSC, 2)",0.0,0.0,0.0
"(CLB, 1)",0.0,0.0,0.0
"(CLB, 2)",0.0,0.0,0.0
"(SPF, 1)",0.0,0.0,0.0
"(SPF, 2)",0.0,0.0,0.0
"(IDP, 1)",0.0,0.0,0.0
"(IDP, 2)",0.0,0.0,0.0


In [ ]:
# transportation shoes
output = {}
for j in destination:
    output[j]=[trans_quantity[(i,j,'Shoes')].varValue for i in plant]
output_df = pd.DataFrame(output, index=plant, columns=destination)
output_df

,STL1,STL2,STL3,STL4,KS1,KS2,COL,SPR
STL,8000.0,6500.0,7500.0,0.0,0.0,0.0,0.0,0.0
KSC,0.0,0.0,0.0,0.0,5200.0,2300.0,0.0,0.0
CLB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SPF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IDP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# transportation boots
output = {}
for j in destination:
    output[j]=[trans_quantity[(i,j,'Boots')].varValue for i in plant]
output_df = pd.DataFrame(output, index=plant, columns=destination)
output_df

,STL1,STL2,STL3,STL4,KS1,KS2,COL,SPR
STL,2200.0,0.0,1200.0,900.0,0.0,0.0,500.0,0.0
KSC,0.0,0.0,0.0,0.0,4500.0,0.0,0.0,0.0
CLB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SPF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IDP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
